In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'

matplotlib.rc("axes", unicode_minus = False)

In [3]:
df1 = pd.read_csv("01 Oxidation.csv")
df2 = pd.read_csv("02 Photo_softbake.csv")
df3 = pd.read_csv("03 Photo_lithograpy.csv")
df4 = pd.read_csv("04 Etching.csv")
df5 = pd.read_csv("05 Ion_Implantation.csv")
df6 = pd.read_csv("06 Inspect.csv")

In [4]:
df = pd.merge(df1, df2, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df3, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df4, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df5, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df6, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])

In [5]:
df.dropna(subset="Thin F2", inplace=True) # 다른 변수들과의 연관성을 찾지못함, 한 행에 여러 열들 결측값 가짐
# pd.set_option('display.max_row', 200)
# pd.set_option('display.max_columns', 200)

In [6]:
df['Ox_Chamber'] = df['Ox_Chamber'].astype('str')
df['photo_soft_Chamber'] = df['photo_soft_Chamber'].astype('str')
df['lithography_Chamber'] = df['lithography_Chamber'].astype('str')
df['Etching_Chamber'] = df['Etching_Chamber'].astype('str')
df['Chamber_Num'] = df['Chamber_Num'].astype('str')
df['path'] = df['Ox_Chamber']+df['photo_soft_Chamber']+df['lithography_Chamber']+df['Etching_Chamber']+df['Chamber_Num']

In [7]:
df.drop(df[df['Oxid_time']<0].index, inplace=True) # 산화시간이 음수
df.drop(df[df['Target']==0].index, inplace=True) # target 값이 0
df.drop(columns = ["Wafer_map","Error_message"],inplace=True) # 웨이퍼맵, 에러메시지

In [8]:
# 시간 데이터 datetime 유형으로 변환
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d-%m-%Y')

# 195이상 = 불량(1), 195미만 = 양품(0)
df.loc[df['Target'] >= 195, '불량_195이상'] =1
df.loc[df['Target'] < 195, '불량_195이상'] =0

In [9]:
df.loc[df['Thin F4']<0, 'Thin F4']=df['Thin F4'].median()
df.loc[df['Flux90s']<0, 'Flux90s']=df['Flux90s'].median()
df.loc[df['Flux160s']<0, 'Flux160s']=df['Flux160s'].median()
df.loc[df['Flux160s']<5, 'Flux160s']=df['Flux160s'].median()

In [10]:
df['Thin F1'].fillna(df['Thin F1'].median(), inplace=True)
df['Thin F3'].fillna(df['Thin F3'].median(), inplace=True)
df['Flux60s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux90s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux480s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux840s'].fillna(df['Flux90s'].median(), inplace=True)

### 불필요한 열 제거

In [11]:
# 불필요한 열 제거 
# df.drop(columns=["No_Die", "Lot_Num", "Wafer_Num"], inplace=True)
df.drop(columns=['Vapor','process','Wavelength'])
df.loc[df['Flux840s'] == 8.137500e+16, 'Flux840s'] = df['Flux840s'].mode()[0]
df.loc[df['Flux480s'] == 8.137500e+16, 'Flux480s'] = df['Flux480s'].mode()[0]

In [24]:
df_new = df[['Temp_OXid','ppm','Pressure','type','Oxid_time',
        'N2_HMDS', 'pressure_HMDS', 'temp_HMDS', 'temp_HMDS_bake',
       'time_HMDS_bake', 'spin1', 'spin2', 'spin3', 'photoresist_bake',
       'temp_softbake', 'time_softbake', 'UV_type', 'Energy_Exposure',       
        'Temp_Etching', 'Source_Power',
       'input_Energy', 'Temp_implantation', 'Furance_Temp', 'RTA_Temp',
       '불량_195이상','path']]

In [25]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 데이터 구성:Series, DataFrame
import pandas as pd
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib

# 데이터 분할:train, test
from sklearn.model_selection import train_test_split
# 분류 Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV

# 분류모델 평가 함수
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

In [26]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

df_raw_x = df_new.drop("불량_195이상", axis = 1)
df_raw_y = df_new["불량_195이상"] 

# MinMaxScaler 객체 생성
numeric_columns = df_raw_x.select_dtypes(include=['number']).columns
scaler = MinMaxScaler()

df_raw_x[numeric_columns] = scaler.fit_transform(df_raw_x[numeric_columns])

df_raw_x = pd.get_dummies(df_raw_x, columns=df_raw_x.select_dtypes(include=['object']).columns)
df_raw_x = df_raw_x.astype({column: int for column in df_raw_x.select_dtypes(include=['bool']).columns})

df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(
    df_raw_x, df_raw_y, test_size = 0.3, random_state = 1234, stratify=df_raw_y) 

In [17]:
columns_to_blank = ['Pressure', 'Temp_OXid', 'temp_softbake', 'photoresist_bake', 
                    'Temp_Etching', 'Temp_implantation', 'spin1', 'spin2', 'Source_Power',
                    'temp_HMDS_bake', 'ppm', 'RTA_Temp']

# 대체할 값의 범위 설정
value_ranges = {
    'Pressure': (0.18, 0.192),
    'Temp_OXid': (1294.58, 1348.47),
    'temp_softbake': (95.28, 96.65),
    'photoresist_bake': (5.14, 5.24),
    'Temp_Etching': (68.15, 68.92),
    'Temp_implantation': (97.74, 100.25),
    'spin1': (492.2, 494),
    'spin2': (5172.1, 5194.13),
    'Source_Power': (49.34, 49.98),
    'temp_HMDS_bake': (205.3, 209.5),
    'ppm': (45.89, 49.91),
    'RTA_Temp': (148, 149)
}

# 테스트 데이터프레임 복사
df_test_x_modified = df_new.copy()

# 선택된 열들의 값을 NaN으로 설정
for column in columns_to_blank:
    df_test_x_modified[column] = np.nan

# 나머지 열들의 값들을 설정된 범위 내에서 랜덤하게 생성하여 대체
for column in df_test_x_modified.columns:
    if column in columns_to_blank:
        min_val, max_val = value_ranges[column]
        df_test_x_modified[column] = np.random.uniform(min_val, max_val, size=len(df_test_x_modified))

# 결과 확인
# df_test_x_modified

,Temp_OXid,ppm,Pressure,Oxid_time,N2_HMDS,pressure_HMDS,temp_HMDS,temp_HMDS_bake,time_HMDS_bake,spin1,...,Source_Power,input_Energy,Temp_implantation,Furance_Temp,RTA_Temp,type_dry,type_wet,UV_type_G,UV_type_H,UV_type_I
1522,1319.168893,49.864011,0.188515,75,11.881,14.924,19.999,209.173554,90.002,492.681209,...,49.907534,32421.295,99.107860,884.0,148.405870,False,True,True,False,False
546,1306.959076,49.688165,0.189469,151,18.779,14.938,19.997,208.197159,90.001,493.160674,...,49.482950,30891.410,100.002314,910.0,148.805356,False,True,False,True,False
412,1336.215075,47.608256,0.181654,220,16.593,14.963,20.001,208.540151,90.000,493.743480,...,49.800994,32300.207,98.465784,921.0,148.345718,True,False,True,False,False
1137,1300.264959,46.960199,0.183181,65,17.434,14.949,20.000,205.910431,90.000,493.239880,...,49.527286,31488.967,98.642474,919.0,148.127516,False,True,False,True,False
93,1302.634891,48.690958,0.181754,56,19.567,14.926,20.002,208.440161,90.001,493.024576,...,49.632907,31732.450,99.407835,912.0,148.946278,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,1315.921492,49.889152,0.180196,203,18.135,15.078,20.002,207.450425,90.002,493.281492,...,49.458837,31756.475,98.131239,922.0,148.856104,True,False,False,False,True
28,1337.370348,48.520762,0.180850,137,13.693,15.059,19.993,207.667497,89.999,493.891509,...,49.707239,32135.618,97.855566,895.0,148.562659,True,False,False,True,False
78,1296.301419,46.772677,0.181270,117,17.587,14.946,19.999,208.699256,90.001,493.872964,...,49.667010,31799.934,98.495079,917.0,148.639353,False,True,True,False,False
237,1332.630311,48.462346,0.188794,112,13.475,15.091,19.995,207.122251,90.001,493.384664,...,49.970162,31068.324,99.813277,903.0,148.119843,False,True,False,False,True


In [19]:
xgb_model = xgb.XGBClassifier(learning_rate=0.5,max_depth=6 ,n_estimators= 242,reg_alpha= 5.551695018865267e-08
                              ,reg_lambda= 7.513887184563936e-08, random_state=1234)
xgb_model.fit(df_train_x, df_train_y)
y_pred = xgb_model.predict(df_test_x_modified)
print("Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))

sum(y_pred)/count(y_pred)

Confusion matrix: 
[[464   0]
 [ 44   0]]
              precision    recall  f1-score   support

         0.0      0.913     1.000     0.955       464
         1.0      0.000     0.000     0.000        44

    accuracy                          0.913       508
   macro avg      0.457     0.500     0.477       508
weighted avg      0.834     0.913     0.872       508



### SMOTE

In [20]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [21]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(df_train_x, df_train_y)
xgb_clf = xgb.XGBClassifier(learning_rate=0.07433674592574852,max_depth=6 ,n_estimators= 181,
                              reg_alpha= 0.08883657957087174,reg_lambda= 0.05472367005012898, 
                              gamma = 0.12620180126392483,
                              random_state=1234)
xgb_clf.fit(X_resampled, y_resampled)

y_pred = xgb_clf.predict(df_test_x_modified)
print("Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))
sum(y_pred)/count(y_pred)

              precision    recall  f1-score   support

           0       0.95      0.98      0.97       464
           1       0.70      0.48      0.57        44

    accuracy                           0.94       508
   macro avg       0.83      0.73      0.77       508
weighted avg       0.93      0.94      0.93       508

